# Perceptrón aplicado a iris

**Lectura del corpus:** $\;$ comprobamos también que las matrices de datos y etiquetas tienen las filas y columnas que toca

In [ ]:
import numpy as np; from sklearn.datasets import load_digits
iris = load_digits(); X = iris.data.astype(np.float16);
y = iris.target.astype(np.uint).reshape(-1, 1);
print(X.shape, y.shape, "\n", np.hstack([X, y])[:5, :])

(1797, 64) (1797, 1) 
 [[ 0.  0.  5. 13.  9.  1.  0.  0.  0.  0. 13. 15. 10. 15.  5.  0.  0.  3.
  15.  2.  0. 11.  8.  0.  0.  4. 12.  0.  0.  8.  8.  0.  0.  5.  8.  0.
   0.  9.  8.  0.  0.  4. 11.  0.  1. 12.  7.  0.  0.  2. 14.  5. 10. 12.
   0.  0.  0.  0.  6. 13. 10.  0.  0.  0.  0.]
 [ 0.  0.  0. 12. 13.  5.  0.  0.  0.  0.  0. 11. 16.  9.  0.  0.  0.  0.
   3. 15. 16.  6.  0.  0.  0.  7. 15. 16. 16.  2.  0.  0.  0.  0.  1. 16.
  16.  3.  0.  0.  0.  0.  1. 16. 16.  6.  0.  0.  0.  0.  1. 16. 16.  6.
   0.  0.  0.  0.  0. 11. 16. 10.  0.  0.  1.]
 [ 0.  0.  0.  4. 15. 12.  0.  0.  0.  0.  3. 16. 15. 14.  0.  0.  0.  0.
   8. 13.  8. 16.  0.  0.  0.  0.  1.  6. 15. 11.  0.  0.  0.  1.  8. 13.
  15.  1.  0.  0.  0.  9. 16. 16.  5.  0.  0.  0.  0.  3. 13. 16. 16. 11.
   5.  0.  0.  0.  0.  3. 11. 16.  9.  0.  2.]
 [ 0.  0.  7. 15. 13.  1.  0.  0.  0.  8. 13.  6. 15.  4.  0.  0.  0.  2.
   1. 13. 13.  0.  0.  0.  0.  0.  2. 15. 11.  1.  0.  0.  0.  0.  0.  1.
  12. 12.  1.  0.  0. 

**Partición del corpus:** $\;$ Creamos un split de iris con un $20\%$ de datos para test y el resto para entrenamiento (training), barajando previamente los datos de acuerdo con una semilla dada para la generación de números aleatorios. Aquí, como en todo código que incluya aleatoriedad (que requiera generar números aleatorios), conviene fijar dicha semilla para poder reproducir experimentos con exactitud.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)
print(X_train.shape, X_test.shape)

(1437, 64) (360, 64)


**Implementación de Perceptrón:** $\;$ devuelve pesos en notación homogénea, $\mathbf{W}\in\mathbb{R}^{(1+D)\times C};\;$ también el número de errores e iteraciones ejecutadas

In [ ]:
def perceptron(X, y, b=0.1, a=1.0, K=200):
    N, D = X.shape; Y = np.unique(y); C = Y.size; W = np.zeros((1+D, C))
    for k in range(1, K+1):
        E = 0
        for n in range(N):
            xn = np.array([1, *X[n, :]])
            cn = np.squeeze(np.where(Y==y[n]))
            gn = W[:,cn].T @ xn; err = False
            for c in np.arange(C):
                if c != cn and W[:,c].T @ xn + b >= gn:
                    W[:, c] = W[:, c] - a*xn; err = True
            if err:
                W[:, cn] = W[:, cn] + a*xn; E = E + 1
        if E == 0:
            break;
    return W, E, k

**Aprendizaje de un clasificador (lineal) con Perceptrón:** $\;$ Perceptrón minimiza el número de errores de entrenamiento (con margen)
$$\mathbf{W}^*=\operatorname*{argmin}_{\mathbf{W}=(\boldsymbol{w}_1,\dotsc,\boldsymbol{w}_C)}\sum_n\;\mathbb{Y}\biggl(\max_{c\neq y_n}\;\boldsymbol{w}_c^t\boldsymbol{x}_n+b \;>\; \boldsymbol{w}_{y_n}^t\boldsymbol{x}_n\biggr)$$

In [ ]:
W, E, k = perceptron(X_train, y_train)
print("Número de iteraciones ejecutadas: ", k)
print("Número de errores de entrenamiento: ", E)
print("Vectores de pesos de las clases (en columnas y en notación homogénea):\n", W);

Número de iteraciones ejecutadas:  106
Número de errores de entrenamiento:  0
Vectores de pesos de las clases (en columnas y en notación homogénea):
 [[-1.420e+02 -2.120e+02 -1.420e+02 -1.430e+02 -1.040e+02 -1.480e+02
  -1.390e+02 -1.410e+02 -1.450e+02 -1.910e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-1.700e+01 -2.600e+01  3.300e+01  5.000e+01 -3.500e+01 -1.300e+01
  -4.200e+01  4.200e+01 -5.700e+01 -1.720e+02]
 [-8.910e+02 -1.029e+03 -9.100e+02 -9.130e+02 -8.400e+02 -5.020e+02
  -1.030e+03 -8.650e+02 -8.160e+02 -9.250e+02]
 [-1.639e+03 -1.522e+03 -1.635e+03 -1.680e+03 -2.119e+03 -1.819e+03
  -1.746e+03 -1.539e+03 -1.805e+03 -1.457e+03]
 [-1.708e+03 -2.382e+03 -1.691e+03 -1.321e+03 -1.808e+03 -1.673e+03
  -1.757e+03 -1.597e+03 -1.682e+03 -1.599e+03]
 [-1.084e+03 -4.850e+02 -9.800e+02 -9.290e+02 -1.280e+03 -6.760e+02
  -1.110e+03 -8.550e+02 -1.067e+03 -1.035e+03]
 [-3.240e+02 -2.790e+02 -2.640e+02 -2.130e+02

**Cálculo de la tasa de error en test:**
El siguiente conjunto de instrucciones calcula la tasa de error para un Perceptron entrenado con b=0.1 y a=1.0 (factor de aprendizaje o 'a') y K=200 iteraciones

In [ ]:
X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])
y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)
err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
print(f"Tasa de error en test: {err_test:.1%}")

Tasa de error en test: 4.4%


In [ ]:
err_test * len(X_test)

16.0

**Ajuste del margen:** $\;$ el siguiente bucle es un experimento que ejecuta 5 veces el algoritmo del Perceptron con 5 valores diferentes de $b$ y valor por defecto $α$=1.0 y K=1000 iteraciones, mostrando el error de entrenamiento para cada valor de $b$.

In [ ]:
for b in (.0, .01, .1, 10, 100):
    W, E, k = perceptron(X_train, y_train, b=b, K=1000)
    print(b, E, k)

0.0 0 106
0.01 0 106
0.1 0 106
10 0 140
100 0 97


**Interpretación de resultados:** $\;$ los datos de entrenamiento no parecen linealmente separables; no está claro que un margen mayor que cero pueda mejorar resultados, sobre todo porque solo tenemos $30$ muestras de test; con margen nulo ya hemos visto que se obtiene un error (en test) del $16.7\%$
-

In [ ]:
min_error = float('inf')
best_combination = None
print('# a b K E k Ete')
for a in (0.1,1.0,10,100,1000,10000):
  for b in (0.0,0.01,0.1,1.0,100,1000):
    for K in (200,500,800,1000):
      W, E, k = perceptron(X_train, y_train, b, a, K)
      X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])
      y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)
      err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
      print('%8.2f %8.2f %6d %6d %6d %8.3f' %(a, b, K, E,k,err_test))
      if err_test < min_error:
                min_error = err_test
                best_combination = (a, b, k)

  print('#------- ------ ---- ---- ---- ----')
print('Mejor combinación con menor error:', best_combination)
print('Menor error encontrado:', min_error)

# a b K E k Ete
    0.10     0.00    200      0    112    0.061
    0.10     0.00    500      0    112    0.061
    0.10     0.00    800      0    112    0.061
    0.10     0.00   1000      0    112    0.061
    0.10     0.01    200      0    106    0.044
    0.10     0.01    500      0    106    0.044
    0.10     0.01    800      0    106    0.044
    0.10     0.01   1000      0    106    0.044
    0.10     0.10    200      0     88    0.042
    0.10     0.10    500      0     88    0.042
    0.10     0.10    800      0     88    0.042
    0.10     0.10   1000      0     88    0.042
    0.10     1.00    200      0     88    0.042
    0.10     1.00    500      0     88    0.042
    0.10     1.00    800      0     88    0.042
    0.10     1.00   1000      0     88    0.042
    0.10   100.00    200      0    172    0.044
    0.10   100.00    500      0    172    0.044
    0.10   100.00    800      0    172    0.044
    0.10   100.00   1000      0    172    0.044
    0.10  1000.00    200

**Ejercicio para realizar en clase:**
---


Escribe el código python necesario para mostrar resultados del error de entrenamiento (E), número de iteraciones empleadas (k) y error de test (err_test) para combinaciones de diferentes valores de:
---

# *   a: (factor de aprendizaje: utiliza valores 0.1, 1.0, 10, 100, 1000, 10000)
# *   b: (margen: utiliza valores 0.0, 0.01, 0.1, 1.0, 100, 1000)
# *   K: (iteraciones: utiliza valores 200, 500, 800 y 1000)


Utiliza la siguientes instrucciones para mostrar la cabecera:
---

`print('#      a        b      K      E      k      Ete');`
---
`print('#-------   ------   ----   ----   ----     ----');`
---

y la siguiente instrucción para mostrar los resultados:

`print('%8.2f %8.2f %6d %6d %6d %8.3f' %(a, b, K, E,k,err_test))`
---





Después de completar la ejecución del bucle, podemos derivar conclusiones valiosas de los resultados obtenidos.

A primera vista, se nota que en general, todos los valores presentan errores de prueba muy similares entre sí. Sin embargo, al profundizar en los resultados, se identifican ciertos patrones significativos. Por ejemplo, los errores de prueba más altos tienden a asociarse con márgenes más bajos, especialmente evidente con el factor de aprendizaje establecido en 0.1. Esta tendencia se mantiene hasta el factor de aprendizaje 10000, donde deja de cumplirse. También es notable que a menor error de entrenamiento corresponde, en general, un menor error de prueba. Asimismo, valores más bajos de factor de aprendizaje suelen coincidir con errores de prueba más bajos, y para un mayor número de iteraciones, se tiende a observar errores de prueba más bajos.

En resumen, la combinación que parece proporcionar el menor error de prueba es un factor de aprendizaje de 0.1, un margen de 0.1 y 1000 iteraciones. Aunque la diferencia es prácticamente imperceptible y se expresa en centésimas, esta configuración se destaca como la más efectiva según los resultados obtenidos.